In [1]:
import numpy as np
import time
import pandas as pd
import requests
from lxml import etree
from tqdm import tqdm_notebook as tqdm
import warnings
import time
import random
import os
import json
import hashlib
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt

In [76]:
# Utilities

def list_house_url(page):
    return 'https://sz.lianjia.com/ershoufang/pg{}/'.format(str(page))

def detai_house_url(idx):
    return 'https://sz.lianjia.com/ershoufang/{}.html'.format(str(idx))

def res_detail_url(idx):
    return 'https://sz.lianjia.com/xiaoqu/{}/'.format(str(idx))

def random_sleep(max_sleep=3, min_sleep=0.2):
    time.sleep(min_sleep + random.random() * max_sleep)
    
def make_header():
    user_agent = [
        'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36',
        'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36 OPR/26.0.1656.60',
        'Mozilla/5.0 (Windows NT 5.1; U; en; rv:1.8.1) Gecko/20061208 Firefox/2.0.0 Opera 9.50',
        'Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; en) Opera 9.50',
        'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:34.0) Gecko/20100101 Firefox/34.0',
        'Mozilla/5.0 (X11; U; Linux x86_64; zh-CN; rv:1.9.2.10) Gecko/20100922 Ubuntu/10.10 (maverick) Firefox/3.6.10',
        'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/534.57.2 (KHTML, like Gecko) Version/5.1.7 Safari/534.57.2',
        'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.71 Safari/537.36',
        'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
        'Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US) AppleWebKit/534.16 (KHTML, like Gecko) Chrome/10.0.648.133 Safari/534.16',
        'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/30.0.1599.101 Safari/537.36',
        'Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko',
        'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/21.0.1180.71 Safari/537.1 LBBROWSER',
        'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; WOW64; Trident/5.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; Media Center PC 6.0; .NET4.0C; .NET4.0E; LBBROWSER)',
        'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; WOW64; Trident/5.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; Media Center PC 6.0; .NET4.0C; .NET4.0E; QQBrowser/7.0.3698.400)',
        'Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; QQDownload 732; .NET4.0C; .NET4.0E)',
        'Mozilla/5.0 (Windows NT 5.1) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.84 Safari/535.11 SE 2.X MetaSr 1.0',
        'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1; Trident/4.0; SV1; QQDownload 732; .NET4.0C; .NET4.0E; SE 2.X MetaSr 1.0)',
    ]
    return {'user-agent': random.choice(user_agent)}

def get_page(url):
    header = make_header()
    r = requests.get(url=url, verify=False, headers=header)
    if r.status_code == 200:
        return etree.HTML(r.text)
    else:
        print(r.status_code)
        
def retrieve(lis):
    try:
        return lis[0]
    except:
        return None
    
def extract_str(string, start, finish):
    if start in string and finish in string:
        idx = string.index(start)
        string = string[idx+len(start):]
        idx = string.index(finish)
        return string[:idx]
    else:
        return None
    
def extract_attr(string, attr):
    start = attr + ':\''
    finish = '\''
    return extract_str(string, start, finish)

In [238]:
# Parse the page

def parse_house_list(page: etree._Element):
    df = pd.DataFrame()
    
    df['house_id'] = page.xpath("//div[@class='info clear']//div[@class='title']/a/@data-housecode")
    df['title'] = page.xpath("//div[@class='info clear']//div[@class='title']/a/text()")
    df['addr_1'] = page.xpath("//div[@class='info clear']//div[@class='positionInfo']//a[1]/text()")
    df['addr_2'] = page.xpath("//div[@class='info clear']//div[@class='positionInfo']//a[2]/text()")

    floor = []
    area = []
    direction = []
    deco = []
    height = []
    age = []
    typ = []
    for i in page.xpath("//div[@class='info clear']//div[@class='houseInfo']/text()"):
        text = i.split('|')
        text = [t.strip() for t in text]
        floor.append(text[0])
        area.append(text[1])
        direction.append(text[2])
        deco.append(text[3])
        height.append(text[4])
        age.append(text[5])
        typ.append(text[6])
    
    df['floor_plan'] = floor
    df['area'] = area
    df['direction'] = direction
    df['decoration'] = deco
    df['height'] = height
    df['age'] = age
    df['type'] = typ
    del floor, area, direction, deco, height, age, typ

    watch = []
    list_time = []
    for i in page.xpath("//div[@class='info clear']//div[@class='followInfo']/text()"):
        text = i.split('/')
        text = [t.strip() for t in text]
        watch.append(text[0])
        list_time.append(text[1])

    df['watch'] = watch
    df['list_time'] = list_time
    
    del watch, list_time

    df['total_price'] = page.xpath("//div[@class='info clear']//div[@class='totalPrice']/span/text()")
    df['unit_price'] = page.xpath("//div[@class='info clear']//div[@class='unitPrice']/@data-price")

    tags = []
    for i in page.xpath("//div[@class='info clear']//div[@class='tag']"):
        tags.append(','.join(i.xpath('./span/text()')))
        
    df['tags'] = tags
    
    return df


def parse_house_detail(page: etree._Element):
    
    def parse_tag(dic, tag):
        if 'tag is_near_subway ' in tag:
            dic['near_subway'] = '1'
        else:
            dic['near_subway'] = '0'

        if 'tag good CLICKDATA VIEWDATA' in tag:
            dic['good'] = '1'
        else:
            dic['good'] = '0'

        if 'tag vr ' in tag:
            dic['vr'] = '1'
        else:
            dic['vr'] = '0'

        if 'tag five ' in tag:
            dic['greater_than_2'] = '1'
        else:
            dic['greater_than_2'] = '0'

        if 'tag is_see_free ' in tag:
            dic['see_anytime'] = '1'
        else:
            dic['see_anytime'] = '0'

        if 'tag taxfree ' in tag:
            dic['greater_than_5'] = '1'
        else:
            dic['greater_than_5'] = '0'
    
    dic = {}
    
    title = retrieve(page.xpath('//h1[@class="main"]/@title'))
    if title is None:
        return None
    dic['title'] = title
    
    pull_off = retrieve(page.xpath('//h1[@class="main"]/span/text()'))
    if pull_off == '已下架':
        dic['pull_off'] = '1'
    else:
        dic['pull_off'] = '0'

    dic['house_id'] = retrieve(page.xpath('//div[@class="houseRecord"]/span[2]/text()'))
    dic['watch'] = retrieve(page.xpath("//span[@id='favCount']/text()"))
#     dic['total_price'] = retrieve(page.xpath('//div[@class="price "]/span[1]/text()'))
#     dic['unit_price'] = retrieve(page.xpath('//div[@class="unitPrice"]/span/text()'))
    dic['community'] = retrieve(page.xpath('//div[@class="communityName"]/a[1]/text()'))
    dic['district'] = retrieve(page.xpath('//div[@class="areaName"]/span[2]/a[1]/text()'))
    dic['addr'] = retrieve(page.xpath('//div[@class="areaName"]/span[2]/a[2]/text()'))
    dic['visit_time'] = retrieve(page.xpath('//div[@class="visitTime"]/span[2]/text()'))
    dic['floor_place'] = retrieve(page.xpath('//div[@class="base"]/div[2]/ul/li[1]/text()'))
    dic['floor'] = retrieve(page.xpath('//div[@class="base"]/div[2]/ul/li[2]/text()'))
    dic['area'] = retrieve(page.xpath('//div[@class="base"]/div[2]/ul/li[3]/text()'))
    dic['floor_structure'] = retrieve(page.xpath('//div[@class="base"]/div[2]/ul/li[4]/text()'))
    dic['area_inside'] = retrieve(page.xpath('//div[@class="base"]/div[2]/ul/li[5]/text()'))
    dic['building_type'] = retrieve(page.xpath('//div[@class="base"]/div[2]/ul/li[6]/text()'))
    dic['facing'] = retrieve(page.xpath('//div[@class="base"]/div[2]/ul/li[7]/text()'))
    dic['building_structure'] = retrieve(page.xpath('//div[@class="base"]/div[2]/ul/li[8]/text()'))
    dic['decoration'] = retrieve(page.xpath('//div[@class="base"]/div[2]/ul/li[9]/text()'))
    dic['stair_to_rooms'] = retrieve(page.xpath('//div[@class="base"]/div[2]/ul/li[10]/text()'))
    dic['elevator'] = retrieve(page.xpath('//div[@class="base"]/div[2]/ul/li[11]/text()'))
    dic['property_duration'] = retrieve(page.xpath('//div[@class="base"]/div[2]/ul/li[12]/text()'))
    dic['list_time'] = retrieve(page.xpath('//div[@class="transaction"]/div[2]/ul/li[1]/span[2]/text()'))
    dic['property_type'] = retrieve(page.xpath('//div[@class="transaction"]/div[2]/ul/li[2]/span[2]/text()'))
    dic['last_transaction'] = retrieve(page.xpath('//div[@class="transaction"]/div[2]/ul/li[3]/span[2]/text()'))
    dic['property_usage'] = retrieve(page.xpath('//div[@class="transaction"]/div[2]/ul/li[4]/span[2]/text()'))
    dic['owned_duration'] = retrieve(page.xpath('//div[@class="transaction"]/div[2]/ul/li[5]/span[2]/text()'))
    dic['property_owner_type'] = retrieve(page.xpath('//div[@class="transaction"]/div[2]/ul/li[6]/span[2]/text()'))
    dic['mortgage'] = retrieve(page.xpath('//div[@class="transaction"]/div[2]/ul/li[7]/span[2]/@title'))
    dic['certificate_photo'] = retrieve(page.xpath('//div[@class="transaction"]/div[2]/ul/li[8]/span[2]/text()'))
    
    tag = page.xpath("//div[@class='tags clear']/div[2]/a/@class")
    parse_tag(dic, tag)
    
    desc = retrieve(page.xpath("//div[@class='baseattribute clear']/div[text() = '周边配套']/following-sibling::*/text()"))
    if desc is not None:
        dic['surrounding_desc'] = desc.strip()
    else:
        dic['surrounding_desc'] = ''
        
    desc = retrieve(page.xpath("//div[@class='baseattribute clear']/div[text() = '小区介绍']/following-sibling::*/text()"))
    if desc is not None:
        dic['community_desc'] = desc.strip()
    else:
        dic['community_desc'] = ''
    
    desc = retrieve(page.xpath('//div[@class="areaName"]/a/text()'))
    if desc is not None:
        dic['addr_desc'] = desc
    else:
        dic['addr_desc'] = ''
        
    desc = retrieve(page.xpath("//div[@class='baseattribute clear']/div[text() = '户型介绍']/following-sibling::*/text()"))
    if desc is not None:
        dic['floor_desc'] = desc.strip()
    else:
        dic['floor_desc'] = ''
        
    desc = retrieve(page.xpath("//div[@class='baseattribute clear']/div[text() = '核心卖点']/following-sibling::*/text()"))
    if desc is not None:
        dic['selling_desc'] = desc.strip()
    else:
        dic['selling_desc'] = ''
        
    desc = retrieve(page.xpath("//div[@class='baseattribute clear']/div[text() = '交通出行']/following-sibling::*/text()"))
    if desc is not None:
        dic['commute_desc'] = desc.strip()
    else:
        dic['commute_desc'] = ''
    
    community_id = retrieve(page.xpath('//div[@class="communityName"]/a[1]/@href'))
    if community_id is not None:
        dic['community_id'] = community_id.split('/')[-2]
    else:
        dic['community_id'] = community_id
        
    floor_detail = []
    for i in page.xpath("//div[@id='infoList']/div"):
        floor_detail.append('/'.join(i.xpath('./div/text()')))
    dic['floor_detail'] = ','.join(floor_detail)
    
    txt = page.xpath('//div[@id="validate-box"]/following-sibling::script/text()')[2]
    dic['position'] = extract_attr(txt, 'resblockPosition')
    dic['total_price'] = extract_attr(txt, 'totalPrice')
    dic['unit_price'] = extract_attr(txt, 'price')
    
    return dic


def parse_res_detail(page: etree._Element, sleep=0.1):
    dic = {}
    dic['title'] = retrieve(page.xpath('//h1[@class="detailTitle"]/text()'))
    dic['addr1'] = retrieve(page.xpath('//div[@class="detailDesc"]/text()'))
    dic['addr2'] = retrieve(page.xpath('//div[@class="fl l-txt"]//a[3]/text()'))
    dic['addr3'] = retrieve(page.xpath('//div[@class="fl l-txt"]//a[4]/text()'))
    dic['price'] = retrieve(page.xpath('//div[@class="xiaoquPrice clear"]/div/span[1]/text()'))
    dic['price_ref'] = retrieve(page.xpath('//div[@class="xiaoquPrice clear"]/div/span[2]/text()'))
    dic['age'] = retrieve(page.xpath('//div[@class="xiaoquInfo"]/div[1]/span[2]/text()'))
    dic['building_type'] = retrieve(page.xpath('//div[@class="xiaoquInfo"]/div[2]/span[2]/text()'))
    dic['management_fee'] = retrieve(page.xpath('//div[@class="xiaoquInfo"]/div[3]/span[2]/text()'))
    dic['management_comp'] = retrieve(page.xpath('//div[@class="xiaoquInfo"]/div[4]/span[2]/text()'))
    dic['development_comp'] = retrieve(page.xpath('//div[@class="xiaoquInfo"]/div[5]/span[2]/text()'))
    dic['num_buildings'] = retrieve(page.xpath('//div[@class="xiaoquInfo"]/div[6]/span[2]/text()'))
    dic['num_rooms'] = retrieve(page.xpath('//div[@class="xiaoquInfo"]/div[7]/span[2]/text()'))
    dic['watch'] = retrieve(page.xpath('//div[@class="followedDesc"]/span/span/text()'))
    
    idx = retrieve(page.xpath('//div[@class="fl l-txt"]//a[5]/@href'))
    dic['res_id'] = idx
    if idx is not None:
        idx = idx.split('/')[-2]
        random_sleep(sleep)
        url = 'https://sz.lianjia.com/xiaoqu/c{}/'.format(idx)
        page_detail = get_page(url)
        dic['num_selled_in_90'] = retrieve(page_detail.xpath('//div[@class="houseInfo"]/a[1]/text()'))
        dic['num_rents'] = retrieve(page_detail.xpath('//div[@class="houseInfo"]/a[2]/text()'))
        dic['num_sell'] = retrieve(page_detail.xpath('//a[@class="totalSellCount"]/span/text()'))
    else:
        dic['num_selled_in_90'] = ''
        dic['num_rents'] = ''
        dic['num_sell'] = ''

    return dic

def parse_house_url(page: etree._Element):
    return page.xpath("//div[@class='info clear']//div[@class='title']/a/@data-housecode")

In [4]:
# The function to crawl the version of only the list
def crawl_main(num_its=100, sleep=3, folder='save'):
    if folder not in os.listdir():
        os.mkdir(folder)

    error_counts = 0
    for i in tqdm(range(1, num_its+1)):
        if error_counts <=100:
            try:
                page = get_page(list_url(i))
                parse_main(page).to_pickle('{}/{}.pkl'.format(folder, str(i)), protocol = -1)
                random_sleep(sleep)
            except:
                error_counts += 1
                pass
        else:
            break
    print('Done')


# Load the result when only crawl the list
def load_house_result(folder):
    
    def parse_list_time(x):
        try:
            if '天' in x:
                return np.int(x[:x.index('天')])
            elif '月' in x:
                return np.int(x[:x.index('月')-1]) * 30
            else:
                return 365
        except:
            return -999
        
    res = []
    for i in os.listdir(folder):
        if 'pkl' in i:
            res.append(pd.read_pickle('{}/{}'.format(folder, str(i))))
    df = pd.concat(res, axis=0).reset_index(drop=True)
    df['total_price'] = df['total_price'].astype(float)
    df['unit_price'] = df['unit_price'].astype(float)
    df['area'] = df['area'].apply(lambda x: x[:x.index('平')]).astype(float)
    df['age'] = df['age'].apply(lambda x: x[:x.index('年')] if '年' in x else -999)
#     df['area'] = df['area'].apply(lambda x: x[:x.index('平')]).astype(float)
#     df['age'] = df['age'].apply(lambda x: x[:x.index('年')]).astype(int)
#     df['watch'] = df['watch'].apply(lambda x: x[:x.index('人')]).astype(int)
    df['list_time'] = df['list_time'].apply(parse_list_time)
    
    return df

In [85]:
def parse_res_detail(page: etree._Element, sleep=0.1):
    
    title = retrieve(page.xpath('//h1[@class="detailTitle"]/text()'))
    if title is None:
        return None
    
    dic = {}
    dic['title'] = title
    dic['addr1'] = retrieve(page.xpath('//div[@class="detailDesc"]/text()'))
    dic['addr2'] = retrieve(page.xpath('//div[@class="fl l-txt"]//a[3]/text()'))
    dic['addr3'] = retrieve(page.xpath('//div[@class="fl l-txt"]//a[4]/text()'))
    dic['price'] = retrieve(page.xpath('//div[@class="xiaoquPrice clear"]/div/span[1]/text()'))
    dic['price_ref'] = retrieve(page.xpath('//div[@class="xiaoquPrice clear"]/div/span[2]/text()'))
    dic['age'] = retrieve(page.xpath('//div[@class="xiaoquInfo"]/div[1]/span[2]/text()'))
    dic['building_type'] = retrieve(page.xpath('//div[@class="xiaoquInfo"]/div[2]/span[2]/text()'))
    dic['management_fee'] = retrieve(page.xpath('//div[@class="xiaoquInfo"]/div[3]/span[2]/text()'))
    dic['management_comp'] = retrieve(page.xpath('//div[@class="xiaoquInfo"]/div[4]/span[2]/text()'))
    dic['development_comp'] = retrieve(page.xpath('//div[@class="xiaoquInfo"]/div[5]/span[2]/text()'))
    dic['num_buildings'] = retrieve(page.xpath('//div[@class="xiaoquInfo"]/div[6]/span[2]/text()'))
    dic['num_rooms'] = retrieve(page.xpath('//div[@class="xiaoquInfo"]/div[7]/span[2]/text()'))
    dic['watch'] = retrieve(page.xpath('//div[@class="followedDesc"]/span/span/text()'))
    
    txt = retrieve(page.xpath('//div[@id="only"]/following-sibling::script[2]/text()'))
    if txt is not None:
        dic['position'] = extract_attr(txt, "resblockPosition")
    else:
        dic['position'] = ''
    
    idx = retrieve(page.xpath('//div[@class="fl l-txt"]//a[5]/@href'))
    dic['num_selled_in_90'] = ''
    dic['num_rents'] = ''
    dic['num_sell'] = ''
    dic['res_id'] = ''
    if idx is not None:
        idx = idx.split('/')[-2]
        dic['res_id'] = idx
        random_sleep(sleep)
        url = 'https://sz.lianjia.com/xiaoqu/c{}/'.format(idx)
        page_detail = get_page(url)
        txt = page_detail.xpath('//div[@class="houseInfo"]/a/text()')
        for i in txt:
            if '成交' in i:
                dic['num_selled_in_90'] = i
            elif '出租' in i:
                dic['num_rents'] = i
            else:
                pass
        dic['num_sell'] = retrieve(page_detail.xpath('//a[@class="totalSellCount"]/span/text()'))


    return dic

In [6]:
def list_res_url(page, area=None, city='sz'):
    if area is not None:
        return 'https://{}.lianjia.com/xiaoqu/{}/pg{}/?from=rec'.format(city, area, str(page))
    return 'https://{}.lianjia.com/xiaoqu/pg{}/?from=rec'.format(city, str(page))

In [112]:
def parse_res_url(page: etree._Element, return_total=False, filter_none=False):
    
    urls = page.xpath('//ul[@class="listContent"]/li/a[@class="img"]/@href')
    total = retrieve(page.xpath('//h2[@class="total fl"]/span/text()'))
    
    if filter_none:
        prices = page.xpath('//div[@class="totalPrice"]/span/text()')
        urls = [urls[i] for i in range(len(urls)) if prices[i] != '暂无']
    
    if return_total:
        total = total.strip()
        return urls, total
    
    return urls

In [111]:
def get_all_res_urls(area=None, city='sz', sleep=3, filter_none=False):
    page = get_page(list_res_url(1, area, city))
    urls, total = parse_res_url(page, return_total=True, filter_none=filter_none)
    random_sleep(sleep)
    its = int(total)//len(urls) + 1
    for i in tqdm(range(2, its+1)):
        page = get_page(list_res_url(i, area, city))
        urls += parse_res_url(page, filter_none=filter_none)
        random_sleep(sleep)
    urls_clean = []
    for i in urls:
        if i not in urls_clean:
            urls_clean.append(i)
    return urls_clean

def crawl_res(urls, sleep=3, batch_size=200, folder='save'):
    if folder not in os.listdir():
        os.mkdir(folder)
    page = get_page(urls[0])
    dic = parse_res_detail(page)
    df = pd.DataFrame(columns=[i for i in dic])
    df = df.append(dic, ignore_index=True)
    random_sleep(sleep)
    batch = 1
    for i in tqdm(range(1, len(urls))):
        try:
            page = get_page(urls[i])
            dic = parse_res_detail(page)
            if (i+1) > batch * batch_size:
                df.to_pickle('{}/{}.pkl'.format(folder, str(batch)), protocol = -1)
                batch += 1
                df = pd.DataFrame(columns=[i for i in dic])            
            df = df.append(dic, ignore_index=True)
            random_sleep(sleep)
        except:
            pass
    df.to_pickle('{}/{}.pkl'.format(folder, str(batch)), protocol = -1)

In [175]:
def get_urls_by_res(search, city='sz', sleep=3):
    p = 1
    url = 'https://{}.lianjia.com/ershoufang/pg{}rs{}/'.format(city, p, search)
    page = get_page(url)
    total = retrieve(page.xpath('//h2[@class="total fl"]/span/text()'))
    if total is None:
        return None
    total = int(total.strip())
    if total == 0:
        return None
    urls = page.xpath('//div[@class="info clear"]/div/a/@href')
    random_sleep(sleep)
    its = int(total)//len(urls) + 1
    for p in tqdm(range(2, its+1)):
        url = 'https://{}.lianjia.com/ershoufang/pg{}rs{}/'.format(city, p, search)
        page = get_page(url)
        urls += page.xpath('//div[@class="info clear"]/div/a/@href')
        random_sleep(sleep)
    urls_clean = []
    for i in urls:
        if i not in urls_clean:
            urls_clean.append(i)
    return urls_clean

def crawl_house(urls, sleep=3, batch_size=200, folder='save'):
    if folder not in os.listdir():
        os.mkdir(folder)
    page = get_page(urls[0])
    dic = parse_house_detail(page)
    df = pd.DataFrame(columns=[i for i in dic])
    df = df.append(dic, ignore_index=True)
    random_sleep(sleep)
    batch = 1
    for i in tqdm(range(1, len(urls))):
        try:
            page = get_page(urls[i])
            dic = parse_house_detail(page)
            if (i+1) > batch * batch_size:
                df.to_pickle('{}/{}.pkl'.format(folder, str(batch)), protocol = -1)
                batch += 1
                df = pd.DataFrame(columns=[i for i in dic])            
            df = df.append(dic, ignore_index=True)
            random_sleep(sleep)
        except:
            pass
    df.to_pickle('{}/{}.pkl'.format(folder, str(batch)), protocol = -1)

In [167]:
url = 'https://{}.lianjia.com/ershoufang/pg{}rs{}/'.format('sz', 1, '深业新岸线')
page = get_page(url)


[]

In [230]:
urls[0]

'https://sz.lianjia.com/ershoufang/105102985219.html'

In [241]:
page = get_page(detai_house_url('105101924121'))
dic = parse_house_detail(page)

In [242]:
dic

{'title': '大阳台，看海，东南向，2009年房，配套好，带装修',
 'pull_off': '0',
 'house_id': '105101924121',
 'watch': '51',
 'community': '深业新岸线三期',
 'district': '宝安区',
 'addr': '宝安中心',
 'visit_time': '只能周末可看',
 'floor_place': '2室1厅1厨1卫',
 'floor': '中楼层 (共25层)',
 'area': '88.88㎡',
 'floor_structure': '暂无数据',
 'area_inside': '73.1㎡',
 'building_type': '塔楼',
 'facing': '东 南',
 'building_structure': '钢混结构',
 'decoration': '精装',
 'stair_to_rooms': '两梯四户',
 'elevator': '有',
 'property_duration': '70年',
 'list_time': '2019-01-09',
 'property_type': '商品房',
 'last_transaction': '2009-03-11',
 'property_usage': '普通住宅',
 'owned_duration': '满五年',
 'property_owner_type': '非共有',
 'mortgage': '无抵押',
 'certificate_photo': '已上传房本照片',
 'near_subway': '1',
 'good': '0',
 'vr': '1',
 'greater_than_2': '0',
 'see_anytime': '1',
 'greater_than_5': '1',
 'surrounding_desc': '',
 'community_desc': '',
 'addr_desc': '近11号线宝安站',
 'floor_desc': '2房，南向户型，在阳台上面可以看到前海湾海景，卧室出20平大阳台，实用，客厅结构合理，可做榻榻米，入户门口还有一个小的空间，可以放鞋子及杂物',
 'selling_desc'

In [153]:
urls = get_all_res_urls('baoanqu', filter_none=True)

In [154]:
crawl_res(urls, sleep=4)

In [155]:
def load_res_result(folder):
        
    res = []
    for i in os.listdir(folder):
        if 'pkl' in i:
            res.append(pd.read_pickle('{}/{}'.format(folder, str(i))))
    df = pd.concat(res, axis=0).reset_index(drop=True)
    df['num_rents'] = df['num_rents'].apply(lambda x: int(x[:x.index('套')]) if '套' in x else 0)
    df['num_rooms'] = df['num_rooms'].apply(lambda x: int(x[:x.index('户')]) if '户' in x else 0)
    df['num_buildings'] = df['num_buildings'].apply(lambda x: int(x[:x.index('栋')]) if '栋' in x else 0)
    df['price'] = df['price'].astype(int)
    df['watch'] = df['watch'].astype(int)
    df['num_sell'] = df['num_sell'].astype(int)
    df['age'] = df['age'].apply(lambda x: int(x[:x.index('年')]) if '年' in x else -999)
    df['num_selled_in_90'] = df['num_selled_in_90'].apply(lambda x: int(x[x.index('交')+1: x.index('套')]) if '套' in x else 0)
    df['addr1'] = df['addr1'].apply(lambda x: x[x.index(')')+1:] if ')' in x else x)
    df['price_ref'] = df['price_ref'].apply(lambda x: x[:x.index('参')] if '参' in x else x)
    
    return df
df = load_res_result('save')

In [226]:
urls = get_urls_by_res('阳光棕榈园')
crawl_house(urls)

In [227]:
def load_house_result(folder):
        
    res = []
    for i in os.listdir(folder):
        if 'pkl' in i:
            res.append(pd.read_pickle('{}/{}'.format(folder, str(i))))
    df = pd.concat(res, axis=0).reset_index(drop=True)
    
    for i in ['pull_off', 'watch', 'total_price', 'unit_price']:
        df[i] = df[i].astype(int)
        
    df['area'] = df['area'].apply(lambda x: float(x[:x.index('㎡')]) if '㎡' in x else 0)
    df['area_inside'] = df['area_inside'].apply(lambda x: float(x[:x.index('㎡')]) if '㎡' in x else 0)
    
    return df
df = load_house_result('save')
df['actual_area_ratio'] = df['area_inside'] / df['area']
ratio = df['actual_area_ratio']
ratio = ratio[ratio!=0]
ratio.mean()

0.8858084085292715

In [215]:
df.sort_values('unit_price',).to_excel('yujingwan.xlsx')

In [228]:
df

,pull_off,house_id,watch,community,district,addr,visit_time,floor_place,floor,area,...,addr_desc,floor_desc,selling_desc,commute_desc,community_id,floor_detail,position,total_price,unit_price,actual_area_ratio
0,0,105102985219,40,阳光棕榈园三期,南山区,前海,一般下班后可看,2室2厅1厨1卫,中楼层 (共11层),73.74,...,,,此房采光好，配套齐全，业主诚意出售,大新地铁口C2出口，小区北门是公交阳光棕榈园站，有229路开往宁水花园总站，234路开往福田...,2413651205958667,"客厅/23.08平米/东/普通窗,卧室A/11.52平米/无/未知窗户类型,卧室B/8.08...","113.918041,22.531379",555,75265,0.000000
1,0,105101486510,108,阳光棕榈园二期,南山区,前海,提前预约随时可看,3室2厅1厨2卫,中楼层 (共10层),98.00,...,近1号线(罗宝线)大新站,,此房满五唯一少税费，临街前海自贸区，近地铁,从小区北门步行606米是地铁1号线罗宝线，大新地铁口C2出口，小区北门是公交阳光棕榈园站，有...,2413651049722581,"客厅/27.89平米/北/落地窗,卧室A/6.33平米/无/未知窗户类型,卧室B/8.93平...","113.919754,22.533528",700,71429,0.819796
2,0,105102054484,33,阳光棕榈园一期,南山区,前海,提前预约随时可看,5室2厅1厨2卫,高楼层 (共11层),183.64,...,近1号线(罗宝线)大新站,顶楼复式三层，五房两卫带两储物间，楼上楼下格局分明，适合一家三代居住。,罕见复式三层，南北通，看小区花园，安静不吵。,小区北门门口是公交车站-阳光棕榈园站，常行车辆有36-42-364-375等，北门直走就是罗...,2411049781865,"客厅/42.9平米/南 北/落地窗,卧室A/9.98平米/无/未知窗户类型,卧室B/12.9...","113.91965036652,22.531635219906",1500,81682,0.000000
3,0,105102363306,37,阳光棕榈园一期,南山区,前海,提前预约随时可看,3室2厅1厨2卫,高楼层 (共6层),109.76,...,近1号线(罗宝线)大新站,,房子满五唯一，客厅阳台朝南 。,阳光棕榈园交通便利，公交车站是深圳交通网络重要站点之一， 经过阳光棕榈园的线路有234路，2...,2411049781865,"客厅/36.52平米/南/普通窗,书房/5.38平米/南/普通窗,卧室A/14.27平米/南...","113.91965036652,22.531635219906",748,68149,0.000000
4,0,105103232448,2,阳光棕榈园二期,南山区,前海,有租户需要预约,2室2厅1厨1卫,中楼层 (共6层),79.39,...,近1号线(罗宝线)大新站,,棕榈园朝西南向两房，楼梯3楼，厅出阳台看小区景观，满五唯一,阳光棕榈园交通便利，公交车站是深圳交通网络重要站点之一， 经过阳光棕榈园的线路有234路，2...,2413651049722581,"客厅/25.5平米/东南 西南/普通窗,卧室A/9.91平米/无/未知窗户类型,卧室B/12...","113.919754,22.533528",600,75577,0.000000
5,0,105102234337,99,阳光棕榈园二期,南山区,前海,提前预约随时可看,4室1厅1厨2卫,低楼层 (共10层),119.36,...,近1号线(罗宝线)大新站,房子户型方正，正南向3+1房，带3个阳台一个大露台，全明格局，进门正对客厅，客厅出阳台， 带...,正南向3+1 客厅出阳台 全明格局 带大露台,小区北门门口是公交车站-南山残链站，常行车辆有36-42-364-375等，北门直走500米...,2413651049722581,"客厅/33.19平米/南/普通窗,书房/4.08平米/无/落地窗,卧室A/8.07平米/无/...","113.919754,22.533528",850,71214,0.000000
6,0,105102259004,84,阳光棕榈园二期,南山区,前海,提前预约随时可看,4室2厅1厨2卫,低楼层 (共6层),133.87,...,近1号线(罗宝线)大新站,此房2005年入伙，南北通透户型，门前有私家花园，进门有入户花园。主卧朝南有阳台，厨房朝北有...,私家花园在1楼，房间均在1.5楼，所以不潮湿，又有入户花园,,2413651049722581,"客厅/37.61平米/南/普通窗,卧室A/14.72平米/无/无窗,卧室B/8.76平米/北...","113.919754,22.533528",920,68724,0.948682
7,0,105102501311,24,阳光棕榈园二期,南山区,前海,只能周末可看,2室2厅1厨1卫,低楼层 (共6层),79.39,...,近1号线(罗宝线)大新站,,棕榈园朝西南向两房，楼梯2楼，厅出阳台看小区景观，满五唯一！,阳光棕榈园交通便利，公交车站是深圳交通网络重要站点之一， 经过阳光棕榈园的线路有234路，2...,2413651049722581,"客厅/26.9平米/南 西/普通窗,卧室A/10.24平米/无/未知窗户类型,卧室B/13....","113.919754,22.533528",600,75577,0.858294
8,0,105103174809,1,阳光棕榈园一期,南山区,前海,提前预约随时可看,3室2厅1厨2卫,低楼层 (共11层),111.24,...,近1号线(罗宝线)大新站,此房为1楼带花园小区中间位置，花园前面很开阔，住着舒服，客厅很大，主卧拓出来了，很大，厨房也...,房子满五年唯一，房子1楼带花园，小区环境很不错，业主诚心卖,阳光棕榈园交通便利，公交车站是深圳交通网络重要站点之一， 经过阳光棕榈园的线路有234路，2...,2411049781865,"客厅/44.21平米/南 西/普通窗,卧室/41.82平米/无/未知窗户类型,卫/1.77平...","113.91965036652,22.531635219906",1236,111112,0.000000
9,0,105102329660,8,阳光棕榈园二期,南山区,前海,提前预约随时可看,4室2厅1厨2卫,中楼层 (共6层),133.75,...,近1号线(罗宝线)大新站,此栋是楼梯房。位于四层。实用率高。实用面积高（房产证面积133.75，126.89）南北通。...,此房在花园中间 安静 南北向 视野通风采光好,,2413651049722581,"客厅/39.55平米/南/落地飘窗,卧室A/14.05平米/无/未知窗户类型,卧室B/11....","113.919754,22.533528",1000,74767,0.948710


In [5]:
# import mysql.connector

# mydb = mysql.connector.connect(
#     host="localhost",
#     user="root",
#     passwd="thinker",
#     database="lianjia"
# )
# cursor = mydb.cursor()

# cursor.execute('DROP Table test')

# cursor.execute('CREATE Table test (id INT AUTO_INCREMENT PRIMARY KEY, name VARCHAR(255), address VARCHAR(255))')

# # Create table
# # cursor.execute('DROP Table house')
# long_chars = ['addr_desc', 'community_desc', 'floor_desc', 'floor_detail', 'selling_desc', 'surrounding_desc']
# short_chars = [i for i in dic if i not in long_chars]
# sql = 'CREATE Table house (id INT AUTO_INCREMENT PRIMARY KEY, '
# sql += ', '.join(["%s VARCHAR(255)"%i for i in short_chars])
# sql += ', '
# sql += ', '.join(["%s TEXT(4096)"%i for i in long_chars])
# sql += ')'

# cursor.execute('SHOW TABLES')

# for i in cursor:
#     print(i)
    
# def insert_house_info(dic, cursor):
#     sql = 'INSERT INTO house (%s) VALUES (%s)'%(
#         ', '.join(short_chars + long_chars),
#         ', '.join('%s' for i in range(len(short_chars + long_chars)))
#     )
#     val = [dic[i] for i in short_chars + long_chars]
#     cursor.execute(sql, val)